In [ ]:
#pip install azure-eventhub

In [ ]:
#pip install azure-eventhub-checkpointstoreblob-aio

In [6]:
# Use this just to get responses from E4 to help code development


import socket
import time
import pylsl
from datetime import datetime 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import requests
import json

from random import randint
%matplotlib qt
#%matplotlib inline

# SELECT DATA TO STREAM
acc = False      # 3-axis acceleration
bvp = False      # Blood Volume Pulse
gsr = True      # Galvanic Skin Response (Electrodermal Activity)
tmp = False      # Temperature
ibi = False      # Interbeat Interval and Heartbeat
bat = False      # Device Battery
tag = False     # Tag taken from the device (by pressing the button)

serverAddress = '127.0.0.1'
serverPort = 28000
bufferSize = 4096

deviceID = 'CDCB11' # 'A03003'

def connect():
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)

    print("Connecting to server")
    s.connect((serverAddress, serverPort))
    print("Connected to server\n")

    print("Devices available:")
    s.send("device_list\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Connecting to device")
    s.send(("device_connect " + deviceID + "\r\n").encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Pausing data receiving")
    s.send("pause ON\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
connect()

time.sleep(1)

def suscribe_to_data():
    if acc:
        print("Suscribing to ACC")
        s.send(("device_subscribe " + 'acc' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bvp:
        print("Suscribing to BVP")
        s.send(("device_subscribe " + 'bvp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if gsr:
        print("Suscribing to GSR")
        s.send(("device_subscribe " + 'gsr' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tmp:
        print("Suscribing to Temp")
        s.send(("device_subscribe " + 'tmp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if ibi:
        print("Suscribing to IBI")
        s.send(("device_subscribe " + 'ibi' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bat:
        print("Suscribing to Batt")
        s.send(("device_subscribe " + 'bat' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tag:
        print("Suscribing to Tag")
        s.send(("device_subscribe " + 'tag' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))

    print("Resuming data receiving")
    s.send("pause OFF\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
suscribe_to_data()

time.sleep(1)

def reconnect():
    print("Reconnecting...")
    connect()
    suscribe_to_data()
    #stream()

def disconnect():
    s.send("device_disconnect\r\n".encode())

#global bvp_timestamp
bvp_timestamp = []
#global bvp_data
bvp_data = []
temp_timestamp = []
temp_data = []
acc_timestamp = []
acc_data = []
gsr_timestamp = []
gsr_data = []
#ibi_timestamp = []
#ibi_data = []


while True:
    response = s.recv(bufferSize).decode("utf-8")
    print(response)
    if "connection lost to device" in response:
        print(response.decode("utf-8"))
        reconnect()
    samples = response.split("\n")
    for i in range(len(samples)-1):
        stream_type = samples[i].split()[0]
        if stream_type == "E4_Bvp":
            bvp_new_timestamp = float(samples[i].split()[1].replace(',','.'))
            bvp_new_data = float(samples[i].split()[2].replace(',','.'))
            bvp_timestamp.append(bvp_new_timestamp)
            bvp_data.append(bvp_new_data)                       
        if stream_type == "E4_Temperature":
            temp_new_timestamp = float(samples[i].split()[1].replace(',','.'))
            temp_new_data = float(samples[i].split()[2].replace(',','.'))
            temp_timestamp.append(temp_new_timestamp)
            temp_data.append(temp_new_data)
        if stream_type == "E4_Acc":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = [int(samples[i].split()[2].replace(',','.')), int(samples[i].split()[3].replace(',','.')), int(samples[i].split()[4].replace(',','.'))]
            acc_timestamp.append(timestamp)
            acc_data.append(data)
        if stream_type == "E4_Gsr":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            gsr_timestamp.append(timestamp)
            gsr_data.append(data)
        if stream_type == "E4_Ibi":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            #ibi_timestamp.append(timestamp)
            #ibi_data.append(data)
            outletIBI.push_sample([data], timestamp=timestamp)
        if stream_type == "E4_Tag":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            outletTag.push_sample([data], timestamp=timestamp)
        if stream_type == "E4_Battery":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            outletBatt.push_sample([data], timestamp=timestamp)
    
"""     myurl = "https://api.powerbi.com/beta/5ee35505-eb8e-4929-937d-645df5013288/datasets/50ad263b-54e0-44ba-93e7-b00cbc39f84d/rows?key=vO%2BdZoHdf6qaU5fg86ZXyj8H%2B4sGYbjCaU6CIvKh8kdC7hnu24Rp38n7785Cvjr4ldnLMGUjL9WuMsZH0CfDsw%3D%3D"
    #date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    data = []

    for i in range(len(temp_data)):
        t = {
            "temp_datetime": datetime.fromtimestamp(temp_timestamp[i]).strftime("%Y-%m-%dT%H:%M:%S"),
            "temp_data": temp_data[i]}
        data.append(t)

    
    r = requests.post(myurl, json.dumps(data))
    print(r.status_code)
    temp_timestamp = []
    temp_data = []
    bvp_timestamp = []
    bvp_data = [] """


Connecting to server
Connected to server

Devices available:
R device_list 1 | CDCB11 Empatica_E4 

Connecting to device
R device_connect OK

Pausing data receiving
R pause ON

Suscribing to GSR
R device_subscribe gsr OK

Resuming data receiving
R pause OFF

E4_Gsr 1627402697.3 0.1999478

E4_Gsr 1627402697.55 0.1973844
E4_Gsr 1627402697.8 0.1999478
E4_Gsr 1627402698.05 0.1986661
E4_Gsr 1627402698.3 0.1999478

E4_Gsr 1627402698.55 0.1973844

E4_Gsr 1627402698.8 0.1973844

E4_Gsr 1627402699.05 0.1973844
E4_Gsr 1627402699.3 0.2012296
E4_Gsr 1627402699.55 0.1999478
E4_Gsr 1627402699.8 0.1986661
E4_Gsr 1627402700.05 0.1999478

E4_Gsr 1627402700.3 0.1986661
E4_Gsr 1627402700.55 0.1999478

E4_Gsr 1627402700.8 0.1961027
E4_Gsr 1627402701.05 0.1999478
E4_Gsr 1627402701.3 0.1961027
E4_Gsr 1627402701.55 0.1986661

E4_Gsr 1627402701.8 0.1961027
E4_Gsr 1627402702.05 0.1999478

E4_Gsr 1627402702.3 0.1973844
E4_Gsr 1627402702.55 0.1999478
E4_Gsr 1627402702.8 0.1999478
E4_Gsr 1627402703.05 0.2025113



KeyboardInterrupt: 

In [7]:
disconnect()

In [2]:
response

'E4_Acc 1627401517.44458 -50 3 39\r\nE4_Acc 1627401517.47583 -50 3 39\r\nE4_Acc 1627401517.50708 -50 3 39\r\nE4_Acc 1627401517.53833 -50 3 39\r\nE4_Acc 1627401517.56958 -50 3 39\r\n'

In [57]:
float(response.split("\r\n")[0].split()[1])#.replace(',','.'))

1627312270.86939

In [58]:
datetime.fromtimestamp(float(response.split("\r\n")[0].split()[1])).strftime("%Y-%m-%dT%H:%M:%S")

'2021-07-26T08:11:10'

In [64]:
##  Old, need to revise so that each sample is added to the batch one by one

def convert_to_json(response):
    response_json = []
    samples = response.split("\r\n")
    for i in range(len(samples)-1):
        stream_type = samples[i].split()[0]
        new_timestamp = datetime.fromtimestamp(float(response.split("\r\n")[0].split()[1])).strftime("%Y-%m-%dT%H:%M:%S")
        new_data = samples[i].split()[2]
        
        new_json = {
            stream_type : new_data,
            "dateTime" : new_timestamp
        }

        response_json.append(new_json)
    response_json = json.dumps(response_json)    
    return(response_json)
       

In [20]:
#updated from above


def convert_to_json(sample):
    
    stream_type = sample.split()[0]
    new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).strftime("%Y-%m-%dT%H:%M:%S")
    new_data = sample.split()[2]
    
    sample_json = {
        stream_type : new_data,
        "dateTime" : new_timestamp
    }

    return(sample_json)


response = 'E4_Bvp 1627312270.86939 0.3150177\r\nE4_Bvp 1627312270.88502 0.4125137\r\nE4_Bvp 1627312270.90064 0.3905487\r\nE4_Bvp 1627312270.91627 0.3139725\r\nE4_Bvp 1627312270.9319 0.2639999\r\nE4_Bvp 1627312270.94752 0.2732544\r\nE4_Bvp 1627312270.96315 0.3248367\r\n'


samples = response.split("\r\n")
for i in range(len(samples)-1):
    sample_json = convert_to_json(samples[i])
    print(sample_json)
    #event_data_batch.add(EventData(sample_json))


{'E4_Bvp': '0.3150177', 'dateTime': '2021-07-26T08:11:10'}
{'E4_Bvp': '0.4125137', 'dateTime': '2021-07-26T08:11:10'}
{'E4_Bvp': '0.3905487', 'dateTime': '2021-07-26T08:11:10'}
{'E4_Bvp': '0.3139725', 'dateTime': '2021-07-26T08:11:10'}
{'E4_Bvp': '0.2639999', 'dateTime': '2021-07-26T08:11:10'}
{'E4_Bvp': '0.2732544', 'dateTime': '2021-07-26T08:11:10'}
{'E4_Bvp': '0.3248367', 'dateTime': '2021-07-26T08:11:10'}


In [5]:
#additional update from above, to account for ACC which has three data points (x,y,z)
import json

def convert_to_json(sample):
    stream_type = sample.split()[0]
    if stream_type == "E4_Acc":
        new_timestamp = float(samples[i].split()[1].replace(',','.'))
        new_data = [int(samples[i].split()[2].replace(',','.')), int(samples[i].split()[3].replace(',','.')), int(samples[i].split()[4].replace(',','.'))]
        sample_json = {"E4_Acc_x" : new_data[0], "E4_Acc_y" : new_data[1], "E4_Acc_z" : new_data[2], "dateTime" : new_timestamp}
        sample_json = json.dumps(sample_json)
    else:
        new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).strftime("%Y-%m-%dT%H:%M:%S")
        new_data = sample.split()[2]
        sample_json = {stream_type : new_data, "dateTime" : new_timestamp}
        sample_json = json.dumps(sample_json)
    return(sample_json)

response = 'E4_Acc 1627401517.44458 -50 3 39\r\nE4_Acc 1627401517.47583 -50 3 39\r\nE4_Acc 1627401517.50708 -50 3 39\r\nE4_Acc 1627401517.53833 -50 3 39\r\nE4_Acc 1627401517.56958 -50 3 39\r\n'
samples = response.split("\r\n")
for i in range(len(samples)-1):
    sample_json = convert_to_json(samples[i])
    print(sample_json)

{"E4_Acc_x": -50, "E4_Acc_y": 3, "E4_Acc_z": 39, "dateTime": 1627401517.44458}
{"E4_Acc_x": -50, "E4_Acc_y": 3, "E4_Acc_z": 39, "dateTime": 1627401517.47583}
{"E4_Acc_x": -50, "E4_Acc_y": 3, "E4_Acc_z": 39, "dateTime": 1627401517.50708}
{"E4_Acc_x": -50, "E4_Acc_y": 3, "E4_Acc_z": 39, "dateTime": 1627401517.53833}
{"E4_Acc_x": -50, "E4_Acc_y": 3, "E4_Acc_z": 39, "dateTime": 1627401517.56958}


# Code chunk below pushes live E4 data to Azure

In [ ]:
## Successful at to recieve data from E4 and simultaneously push it to Azure

import socket
import time
import pylsl
from datetime import datetime 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import requests
import json
from azure.eventhub import EventHubProducerClient, EventData
import os
import time

from random import randint
%matplotlib qt
#%matplotlib inline

session_ID = "tb_" + str(time.time())
participant = "Tony Banks"
participant_ID = "001"

# SELECT DATA TO STREAM
acc = True      # 3-axis acceleration
bvp = True     # Blood Volume Pulse
gsr = True      # Galvanic Skin Response (Electrodermal Activity)
tmp = True      # Temperature
ibi = True      # Interbeat Interval and Heartbeat
bat = False      # Device Battery
tag = False     # Tag taken from the device (by pressing the button)

serverAddress = '127.0.0.1'
serverPort = 28000
bufferSize = 4096

deviceID = 'CDCB11' # 'A03003'

def connect():
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)

    print("Connecting to server")
    s.connect((serverAddress, serverPort))
    print("Connected to server\n")

    print("Devices available:")
    s.send("device_list\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Connecting to device")
    s.send(("device_connect " + deviceID + "\r\n").encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Pausing data receiving")
    s.send("pause ON\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
connect()

time.sleep(1)

def suscribe_to_data():
    if acc:
        print("Suscribing to ACC")
        s.send(("device_subscribe " + 'acc' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bvp:
        print("Suscribing to BVP")
        s.send(("device_subscribe " + 'bvp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if gsr:
        print("Suscribing to GSR")
        s.send(("device_subscribe " + 'gsr' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tmp:
        print("Suscribing to Temp")
        s.send(("device_subscribe " + 'tmp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if ibi:
        print("Suscribing to IBI")
        s.send(("device_subscribe " + 'ibi' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bat:
        print("Suscribing to Batt")
        s.send(("device_subscribe " + 'bat' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tag:
        print("Suscribing to Tag")
        s.send(("device_subscribe " + 'tag' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))

    print("Resuming data receiving")
    s.send("pause OFF\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
suscribe_to_data()

time.sleep(1)

def reconnect():
    print("Reconnecting...")
    connect()
    suscribe_to_data()
    #stream()

def disconnect():
    s.send("device_disconnect\r\n".encode())

def convert_to_json(sample):
    stream_type = sample.split()[0]
    if stream_type == "E4_Acc":
        new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).strftime("%Y-%m-%dT%H:%M:%S")
        new_data = [int(samples[i].split()[2].replace(',','.')), int(samples[i].split()[3].replace(',','.')), int(samples[i].split()[4].replace(',','.'))]
        sample_jsonX = {"stream_type" : "E4_Acc_X", "Value" : new_data[0], "dateTime" : new_timestamp, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_jsonY = {"stream_type" : "E4_Acc_Y", "Value" : new_data[1], "dateTime" : new_timestamp, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_jsonZ = {"stream_type" : "E4_Acc_Z", "Value" : new_data[2], "dateTime" : new_timestamp, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_jsons = [sample_jsonX, sample_jsonY, sample_jsonZ]
        sample_json = json.dumps(sample_json)    #might have issues from trying to batch more than 1 json at a time later in the code
    else:
        new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).strftime("%Y-%m-%dT%H:%M:%S")
        new_data = sample.split()[2]
        sample_json = {"stream_type" : stream_type, "Value" : new_data, "dateTime" : new_timestamp, "Session_ID" : session_ID, "Participant" : participant, "Participant_ID" : participant_ID}
        sample_json = json.dumps(sample_json)
    return(sample_json)

eventhub_name = 'eventhub'
client = EventHubProducerClient.from_connection_string(os.environ.get('CONNECTION_STRING'), eventhub_name=eventhub_name)

try:
    while True:
        response = s.recv(bufferSize).decode("utf-8")
        #print(response)
        if "connection lost to device" in response:
            print(response.decode("utf-8"))
            reconnect()
        event_data_batch = client.create_batch()
        samples = response.split("\r\n")
        for i in range(len(samples)-1):
            sample_json = convert_to_json(samples[i])
            print(sample_json)
            event_data_batch.add(EventData(sample_json))
        client.send_batch(event_data_batch)
except KeyboardInterrupt:
    print('interrupted!')
    disconnect()



In [6]:
samples = [{"E4_Acc_x": -4, "E4_Acc_y": 2, "E4_Acc_z": 64, "ACC_dateTime": "2021-07-27T10:17:09", "Session_ID": "tb_1627406228.917435", "Participant": "Tony Banks", "Participant_ID": "001"},{"E4_Acc_x": -4, "E4_Acc_y": 3, "E4_Acc_z": 64, "ACC_dateTime": "2021-07-27T10:17:10", "Session_ID": "tb_1627406228.917435", "Participant": "Tony Banks", "Participant_ID": "001"},{"E4_Acc_x": -4, "E4_Acc_y": 3, "E4_Acc_z": 64, "ACC_dateTime": "2021-07-27T10:17:10", "Session_ID": "tb_1627406228.917435", "Participant": "Tony Banks", "Participant_ID": "001"}]

In [7]:
import json

json.dumps(samples)

'[{"E4_Acc_x": -4, "E4_Acc_y": 2, "E4_Acc_z": 64, "ACC_dateTime": "2021-07-27T10:17:09", "Session_ID": "tb_1627406228.917435", "Participant": "Tony Banks", "Participant_ID": "001"}, {"E4_Acc_x": -4, "E4_Acc_y": 3, "E4_Acc_z": 64, "ACC_dateTime": "2021-07-27T10:17:10", "Session_ID": "tb_1627406228.917435", "Participant": "Tony Banks", "Participant_ID": "001"}, {"E4_Acc_x": -4, "E4_Acc_y": 3, "E4_Acc_z": 64, "ACC_dateTime": "2021-07-27T10:17:10", "Session_ID": "tb_1627406228.917435", "Participant": "Tony Banks", "Participant_ID": "001"}]'

In [8]:
import time

"tb_" + str(time.time())

'tb_1627405727.2363806'

In [25]:
##  Sucessfully sending one batch containing JSON of all info from a single "response" from E4
from datetime import datetime 
import json
from azure.eventhub import EventHubProducerClient, EventData
import os


response = 'E4_Bvp 1627312270.86939 0.3150177\r\nE4_Bvp 1627312270.88502 0.4125137\r\nE4_Bvp 1627312270.90064 0.3905487\r\nE4_Bvp 1627312270.91627 0.3139725\r\nE4_Bvp 1627312270.9319 0.2639999\r\nE4_Bvp 1627312270.94752 0.2732544\r\nE4_Bvp 1627312270.96315 0.3248367\r\n'

def convert_to_json(sample):
    
    stream_type = sample.split()[0]
    new_timestamp = datetime.fromtimestamp(float(sample.split()[1])).strftime("%Y-%m-%dT%H:%M:%S")
    new_data = sample.split()[2]
    
    sample_json = {
        stream_type : new_data,
        "dateTime" : new_timestamp
    }
    sample_json = json.dumps(sample_json)
    return(sample_json)

from azure.eventhub import EventHubProducerClient, EventData

eventhub_name = 'eventhub'
client = EventHubProducerClient.from_connection_string(os.environ.get('CONNECTION_STRING'), eventhub_name=eventhub_name)

event_data_batch = client.create_batch()

samples = response.split("\r\n")

for i in range(len(samples)-1):
    sample_json = convert_to_json(samples[i])
    print(sample_json)
    event_data_batch.add(EventData(sample_json))


client.send_batch(event_data_batch)


{"E4_Bvp": "0.3150177", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.4125137", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.3905487", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.3139725", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.2639999", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.2732544", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.3248367", "dateTime": "2021-07-26T08:11:10"}


In [26]:
samples = response.split("\r\n")

for i in range(len(samples)-1):
    sample_json = convert_to_json(samples[i])
    print(sample_json)

{"E4_Bvp": "0.3150177", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.4125137", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.3905487", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.3139725", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.2639999", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.2732544", "dateTime": "2021-07-26T08:11:10"}
{"E4_Bvp": "0.3248367", "dateTime": "2021-07-26T08:11:10"}


# Code below works to send message to eventhub.  Eventhub sends batch of messages to blobstorage

In [36]:
#Sending multiples of the same message, probably because of the while/try is creating duplicates before sending the batch

from azure.eventhub import EventHubProducerClient, EventData

eventhub_name = 'eventhub'
client = EventHubProducerClient.from_connection_string(os.environ.get('CONNECTION_STRING'), eventhub_name=eventhub_name)

event_data_batch = client.create_batch()
can_add = True
while can_add:
    try:
        event_data_batch.add(EventData('{"messageId":3,"deviceId":"Python Client","temperature":43.22222,"humidity":22.11111}'))  #JSON string
    except ValueError:
        can_add = False  # EventDataBatch object reaches max_size.

with client:
    client.send_batch(event_data_batch)

client.close()

In [38]:
client.close()

In [9]:
import os

os.environ['CONNECTION_STRING'] = 'Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=eventpolicy;SharedAccessKey=gR5uUifVoEXQpY57GvQMvZ/YUpAqA87A/hSo3JgdgC8=;EntityPath=eventhub'

In [2]:
os.environ.get('CONNECTION_STRING')

'Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=eventpolicy;SharedAccessKey=gR5uUifVoEXQpY57GvQMvZ/YUpAqA87A/hSo3JgdgC8=;EntityPath=eventhub'

In [4]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData


    # Create a producer client to send messages to the event hub.
    # Specify a connection string to your event hubs namespace and
    # the event hub name.
producer = EventHubProducerClient.from_connection_string(conn_str="pythonEventHb-Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=AQ/07wNguW0MtGKasw8IXcIoq9L3P44zwrFDsMLSWOg=", eventhub_name="eventhub")
async with producer:
    # Create a batch.
    event_data_batch = await producer.create_batch()

    # Add events to the batch.
    event_data_batch.add(EventData('First event '))
    event_data_batch.add(EventData('Second event'))
    event_data_batch.add(EventData('Third event'))

    # Send the batch of events to the event hub.
    await producer.send_batch(event_data_batch)


ValueError: Connection string is either blank or malformed.